In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi model
w_model = gp.Model("PolygonOptimizer")

# Define polygon vertices with axis-aligned constraints
num_vertices = 1000
x = w_model.addVars(num_vertices, lb=0, ub=10000, name="x")
y = w_model.addVars(num_vertices, lb=0, ub=10000, name="y")
edge_type = w_model.addVars(num_vertices, vtype=GRB.BINARY, name="edge_type")  # 0=vertical, 1=horizontal

# Axis-alignment constraints
for i in range(num_vertices):
    next_i = (i + 1) % num_vertices
    w_model.addConstr((edge_type[i] == 1) >> (x[i] == x[next_i]))  # Horizontal edge
    w_model.addConstr((edge_type[i] == 0) >> (y[i] == y[next_i]))  # Vertical edge

# Closed polygon constraint
w_model.addConstr(x[0] == x[num_vertices - 1])
w_model.addConstr(y[0] == y[num_vertices - 1])



# Define binary helper variables indexed by (i, j, type)
b_varss = w_model.addVars(998000 * 2, vtype=GRB.BINARY, name="b")

# Constants
MM = 10000  # Big-M constant
epsilon = 1e-6  

# Counter for binary variable index
cnt = 0  

# Add constraints
for i in range(1, 998):
    for j in range(i + 2, 1000):
        ximi = x[i - 1]
        xjmi = x[j - 1]
        xi = x[i]
        xj = x[j]

        yimi = y[i - 1]
        yjmi = y[j - 1]
        yi = y[i]
        yj = y[j]

        # Define auxiliary variable to avoid division
        num_x = (yjmi - yimi) * (xi - ximi) - (xjmi - ximi) * (yi - yimi)
        den_x = (xj - xjmi) * (yi - yimi) - (xi - ximi) * (yj - yjmi)

        num_y = (yjmi - yimi) * (xj - xjmi) - (xjmi - ximi) * (yj - yjmi)
        den_y = den_x  # Same denominator

        # Use Big-M to enforce strict inequalities
        w_model.addConstr(num_x <= -epsilon * den_x + MM * b_varss[cnt], name=f"x_lt_0_{i}_{j}")
        w_model.addConstr(num_x >= (1 + epsilon) * den_x - MM * (1 - b_varss[cnt]), name=f"x_gt_1_{i}_{j}")
        cnt += 1

        w_model.addConstr(num_y <= -epsilon * den_y + MM * b_varss[cnt], name=f"y_lt_0_{i}_{j}")
        w_model.addConstr(num_y >= (1 + epsilon) * den_y - MM * (1 - b_varss[cnt]), name=f"y_gt_1_{i}_{j}")
        cnt += 1

# Update model
w_model.update()


# Simplified point inclusion using vertical edges only
def create_point_constraints(px, py, coeff):
    cross_count = 0
    for i in range(num_vertices):
        x1, y1 = x[i], y[i]
        x2, y2 = x[(i + 1) % num_vertices], y[(i + 1) % num_vertices]

        # Only consider vertical edges (edge_type == 0)
        is_vertical = w_model.addVar(vtype=GRB.BINARY, name=f"is_vertical_{i}")
        w_model.addConstr(is_vertical == 1 - edge_type[i])  # 1 if vertical, 0 if horizontal

        # Create crossing indicator
        cross = w_model.addVar(vtype=GRB.BINARY, name=f"cross_{i}")

        # Define binary conditions
        x_cond = w_model.addVar(vtype=GRB.BINARY, name=f"x_cond_{i}")
        y1_cond = w_model.addVar(vtype=GRB.BINARY, name=f"y1_cond_{i}")
        y2_cond = w_model.addVar(vtype=GRB.BINARY, name=f"y2_cond_{i}")

        # Big-M constraints to enforce conditions
        M = 10000  # Large enough constant
        w_model.addConstr(x1 - px >= -M * (1 - x_cond))  # x1 >= px -> x_cond = 1
        w_model.addConstr(y2 - py >= -M * (1 - y2_cond))  # y2 >= py -> y2_cond = 1
        w_model.addConstr(y1 - py <= M * (1 - y1_cond))  # y1 <= py -> y1_cond = 1

        # Ensure cross is 1 only if all conditions hold
        w_model.addConstr(cross <= is_vertical)
        w_model.addConstr(cross <= x_cond)
        w_model.addConstr(cross <= y1_cond)
        w_model.addConstr(cross <= y2_cond)
        w_model.addConstr(cross >= is_vertical + x_cond + y1_cond + y2_cond - 3)

        cross_count += cross

    # Odd number of crossings = inside
    inside = w_model.addVar(vtype=GRB.BINARY, name="inside")

    # Handle modulo using binary constraints
    even_cross = w_model.addVar(vtype=GRB.BINARY, name="even_cross")
    w_model.addConstr(cross_count == 2 * even_cross + inside)  # Ensures modulo behavior

    return inside * coeff

# Build objective function
total_score = 0

# Add crystals with positive weight
for a, b, c in crystals:
    total_score += create_point_constraints(a, b, c)

# Add mines with negative weight
for a, b, c in mines:
    total_score -= create_point_constraints(a, b, c)

# Solve parameters
w_model.setObjective(total_score, GRB.MAXIMIZE)
w_model.Params.NonConvex = 2
w_model.Params.IterationLimit = 100  # Limits the number of iterations

w_model.Params.TimeLimit = 150 # 5-minute time limit

# Solve and output
w_model.optimize()

if model.Status == GRB.INTERRUPTED or model.SolCount > 0:
    print(f"Optimal Score: {w_model.objVal}")
    # Output first 10 vertices for verification
    for i in range(1000):
        print(f"Vertex {i}: ({x[i].X:.1f}, {y[i].X:.1f})")
else:
    print("No valid solution found")